In [2]:
import csv
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
from time import sleep
from bs4 import SoupStrainer
import requests
import lxml
import random
from datetime import datetime
from itertools import cycle

In [22]:
def get_ips():
    ip_list = []
    with open('myproxylist.txt', 'r') as file:
        for line in file:
            proxy = line.strip()
            ip = proxy.split(':')[0]
            ip_list.append(ip)
    return ip_list

In [27]:
def append_dict_as_row(file_name, dict_of_elem):
    # Specify field names
    field_names = ['Url', 'Headline', 'ProxyIP', 'Publication Date', 'Content', 'Image URL']
    
    # Open the CSV file in append mode and create a CSV writer object
    with open(file_name, 'a', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.DictWriter(csvfile, fieldnames=field_names)
        
        # Check if the file is empty (no existing header)
        file_empty = csvfile.tell() == 0
        
        # If the file is empty, write the header row
        if file_empty:
            csvwriter.writeheader()
        
        # Write the values of the dictionary as a row in the CSV file
        csvwriter.writerow(dict_of_elem)

In [28]:
count = 0

ip_list = get_ips()

# use a proxy pool and cycle  
proxy_pool = cycle(ip_list)
proxy = next(proxy_pool)

def fetch_news_bbg(link):
    global count

    global proxy_pool
    global proxy

    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(link, headers=headers, proxies={"http": proxy})
    soup = bs(response.text, 'html.parser')

    for links in soup.find_all('div', class_='c-compact-river__entry'):
        headline = links.find('h2', class_='c-entry-box--compact__title')

        if 'lionel' in headline.text.lower() or 'messi' in headline.text.lower():

            count += 1

            a_tag = links.find('a')['href']

            time = links.find('time', class_='c-byline__item')

            proxy = next(proxy_pool)

            response_inside = requests.get(a_tag, headers=headers, proxies={"http": proxy})
            soup_inside = bs(response_inside.text, 'html.parser')

            try:
                image = soup_inside.find('picture', class_='c-picture').find('img')['srcset'].split(',')[0][:-4]
            except:
                image = 'No Image'
            content = soup_inside.find('div', class_='l-col__main').find_all('p')
            content = '\n'.join([p.get_text(strip=True) for p in content])

            print(f'Article Number: {count}, ProxyIP: {proxy} Content: {content[:20]}, Headline: {headline.text}, Link: {a_tag}, Publication Date: {time.text.strip()}, Image URL: {image}')
            append_dict_as_row('dataset_bbg.csv', {'Url': a_tag, 'Headline': headline.text, 'ProxyIP': proxy, 'Publication Date': time.text.strip(), 'Content': content, 'Image URL': image})

In [29]:
base_url = 'https://www.barcablaugranes.com/search?page='
string_added = '&q=lionel%20messi'

num_pages = 100

for i in range(1, num_pages+1):
    url = base_url + str(i) + string_added
    fetch_news_bbg(url)
    proxy = next(proxy_pool)
    sleep(2)

    print(f'Page {i} done')

Article Number: 1, ProxyIP: 185.199.229.156 Content: Barcelona star Lamin, Headline: WATCH: Yamal scores Messi-esque goal for Barca vs Athletic, Link: https://www.barcablaugranes.com/2024/1/24/24049622/watch-lamine-yamal-scores-lionel-messi-esque-goal-for-barcelona-against-athletic, Publication Date: Jan 24, 2024, Image URL: https://cdn.vox-cdn.com/thumbor/MDWLmC59VjMp-tJSS9ncrq7q0yc=/0x0:4282x2855/320x213/filters:focal(1932x410:2616x1094)/cdn.vox-cdn.com/uploads/chorus_image/image/73082216/1950954917.0.jpg 
Article Number: 2, ProxyIP: 185.199.228.220 Content: FC Barcelonapresiden, Headline: Barcelona president rules out one-match loan for Lionel Messi, Link: https://www.barcablaugranes.com/2023/12/20/24008283/barcelona-president-rules-out-one-match-loan-for-lionel-messi, Publication Date: Dec 20, 2023, Image URL: https://cdn.vox-cdn.com/thumbor/W1gfnSiwmfYZPBPx9105Q7PR_Q8=/0x0:3460x5188/320x213/filters:focal(1341x867:1893x1419)/cdn.vox-cdn.com/uploads/chorus_image/image/72981449/17757

In [30]:
base_url = 'https://www.barcablaugranes.com/search?page='
string_added = '&q=lionel%20messi'

num_pages = 150

for i in range(101, num_pages+1):
    url = base_url + str(i) + string_added
    fetch_news_bbg(url)
    proxy = next(proxy_pool)
    sleep(2)

    print(f'Page {i} done')

Article Number: 779, ProxyIP: 45.94.47.66 Content: Jordi Alba has repor, Headline: Jordi Alba offered chance to join Messi in Miami - report, Link: https://www.barcablaugranes.com/2023/6/14/23760491/jordi-alba-offered-chance-to-join-lionel-messi-at-inter-miami-report, Publication Date: Jun 14, 2023, Image URL: https://cdn.vox-cdn.com/thumbor/VjOevJcOBGsQqxcSvh3jk-Q1810=/0x0:1750x1207/320x213/filters:focal(543x80:823x360)/cdn.vox-cdn.com/uploads/chorus_image/image/72368767/1231635601.0.jpg 
Article Number: 780, ProxyIP: 38.154.227.167 Content: FC Barcelonaare hope, Headline: Lionel Messi renewal could be announced next week, Link: https://www.barcablaugranes.com/2021/7/26/22594581/lionel-messi-renewal-could-be-announced-next-week, Publication Date: Jul 26, 2021, Image URL: https://cdn.vox-cdn.com/thumbor/aX5A8fAjFeXfU7J6fqRY5R9ZNyM=/0x0:4500x3214/320x213/filters:focal(2029x529:2749x1249)/cdn.vox-cdn.com/uploads/chorus_image/image/69635004/1232945457.0.jpg 
Article Number: 781, ProxyIP: 

In [2]:
import pandas as pd

df_bbc = pd.read_csv('dataset_bbc.csv')
df_toi = pd.read_csv('dataset_toi.csv')
df_apn = pd.read_csv('dataset_apn.csv')
df_bbg = pd.read_csv('dataset_bbg.csv')
df_90Min = pd.read_csv('dataset_90Min.csv')

combined_dataset_ = pd.concat([df_bbg, df_toi, df_apn, df_bbc, df_90Min], ignore_index=True)
combined_dataset_.to_csv('combined_dataset_.csv', index=False)

print(combined_dataset_.shape)

(2867, 6)


In [3]:
combined_df = pd.read_csv('combined_dataset_.csv')

In [6]:
combined_df

,Url,Headline,ProxyIP,Publication Date,Content,Image URL
0,https://www.barcablaugranes.com/2024/1/24/2404...,WATCH: Yamal scores Messi-esque goal for Barca...,185.199.229.156,"Jan 24, 2024",Barcelona star Lamine Yamal scored a brilliant...,https://cdn.vox-cdn.com/thumbor/MDWLmC59VjMp-t...
1,https://www.barcablaugranes.com/2023/12/20/240...,Barcelona president rules out one-match loan f...,185.199.228.220,"Dec 20, 2023",FC Barcelonapresident Joan Laporta has ruled o...,https://cdn.vox-cdn.com/thumbor/W1gfnSiwmfYZPB...
2,https://www.barcablaugranes.com/2023/12/22/240...,"Barca reunion continues as Suarez joins Messi,...",185.199.231.45,"Dec 22, 2023",Luis Suarez has become the latest former Barce...,https://cdn.vox-cdn.com/thumbor/eh4NUh7GSdw_El...
3,https://www.barcablaugranes.com/2023/12/5/2398...,Lionel Messi confirms returning to Barcelona w...,188.74.183.10,"Dec 5, 2023",Lionel Messi has confirmed that going back toF...,https://cdn.vox-cdn.com/thumbor/wuXvEFdpfgqbAq...
4,https://www.barcablaugranes.com/2023/12/5/2398...,"Messi has always been my idol, says Lamine Yamal",188.74.210.21,"Dec 5, 2023",Barcelona wonderkid Lamine Yamal has regularly...,https://cdn.vox-cdn.com/thumbor/c7o6yEkfyg5RQe...
...,...,...,...,...,...,...
2862,https://www.90min.com/posts/on-this-day-in-foo...,On This Day in Football History - 28 May: Mess...,188.74.210.21,"May 28, 2020",On this Day in football history - May 28: Tott...,https://images2.minutemediacdn.com/image/uploa...
2863,https://www.90min.com/posts/not-completing-the...,Not Completing the 2019/20 Football Season Cou...,188.74.210.21,"May 7, 2020",Player values could plummet by £8.7bn across E...,https://images2.minutemediacdn.com/image/uploa...
2864,https://www.90min.com/posts/lionel-messi-wins-...,Lionel Messi Wins the Ballon d'Or... Again: Se...,188.74.210.21,"May 5, 2020",Season Highlight: Messi Wins the Ballon D'or.....,https://images2.minutemediacdn.com/image/uploa...
2865,https://www.90min.com/posts/7-of-lionel-messi-...,7 of Lionel Messi's Milestone Goals,188.74.210.21,"Jun 24, 2020",7 of Lionel Messi's milestone goals for Barcel...,https://images2.minutemediacdn.com/image/uploa...


In [5]:
from transformers import BertTokenizer
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import random 

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
# Convert to string and lowercase
combined_df['Content'] = combined_df['Content'].astype(str)
combined_df['Content'] = combined_df['Content'].str.lower()

# Remove non-alphanumeric characters
combined_df['Content'] = combined_df['Content'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

# Remove extra whitespaces
combined_df['Content'] = combined_df['Content'].apply(lambda x: re.sub(r'\s+', ' ', x))

# Remove stopwords
stop_words = set(stopwords.words('english'))
combined_df['Content'] = combined_df['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [9]:
def augment_text(text):
    tokens = text.split()
    augmented_tokens = []
    for token in tokens:
        synonyms = get_synonyms(token)
        if synonyms:
            augmented_tokens.append(random.choice(synonyms))
        else:
            augmented_tokens.append(token)
    return ' '.join(augmented_tokens)

In [10]:
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return list(set(synonyms))

In [11]:
combined_df['Augmented_Content'] = combined_df['Content'].apply(augment_text)

In [12]:
combined_df.head(10)

,Url,Headline,ProxyIP,Publication Date,Content,Image URL,Augmented_Content
0,https://www.barcablaugranes.com/2024/1/24/2404...,WATCH: Yamal scores Messi-esque goal for Barca...,185.199.229.156,"Jan 24, 2024",barcelona star lamine yamal scored brilliant g...,https://cdn.vox-cdn.com/thumbor/MDWLmC59VjMp-t...,Barcelona star_topology lamine yamal score sup...
1,https://www.barcablaugranes.com/2023/12/20/240...,Barcelona president rules out one-match loan f...,185.199.228.220,"Dec 20, 2023",fc barcelonapresident joan laporta ruled repor...,https://cdn.vox-cdn.com/thumbor/W1gfnSiwmfYZPB...,fc barcelonapresident joan laporta prevail acc...
2,https://www.barcablaugranes.com/2023/12/22/240...,"Barca reunion continues as Suarez joins Messi,...",185.199.231.45,"Dec 22, 2023",luis suarez become latest former barcelona pla...,https://cdn.vox-cdn.com/thumbor/eh4NUh7GSdw_El...,luis suarez become former early Barcelona thes...
3,https://www.barcablaugranes.com/2023/12/5/2398...,Lionel Messi confirms returning to Barcelona w...,188.74.183.10,"Dec 5, 2023",lionel messi confirmed going back tofc barcelo...,https://cdn.vox-cdn.com/thumbor/wuXvEFdpfgqbAq...,lionel messi support leaving game tofc barcelo...
4,https://www.barcablaugranes.com/2023/12/5/2398...,"Messi has always been my idol, says Lamine Yamal",188.74.210.21,"Dec 5, 2023",barcelona wonderkid lamine yamal regularly lik...,https://cdn.vox-cdn.com/thumbor/c7o6yEkfyg5RQe...,Barcelona wonderkid lamine yamal on_a_regular_...
5,https://www.barcablaugranes.com/2023/12/1/2398...,Lionel Messi opens up on feud with Robert Lewa...,45.155.68.129,"Dec 1, 2023",barcelona legend lionel messi admitted pretty ...,https://cdn.vox-cdn.com/thumbor/OXccKTnc9uTI6R...,Barcelona caption lionel messi acknowledge som...
6,https://www.barcablaugranes.com/2023/11/14/239...,Barcelona target Messinho shines at U17 World Cup,154.95.36.199,"Nov 14, 2023",estevo willian also known nickname messinho sh...,https://cdn.vox-cdn.com/thumbor/GgY_lYAzUiRiKu...,estevo willian as_well fuck cognomen messinho ...
7,https://www.barcablaugranes.com/2023/11/4/2394...,Messi responds to Barcelona loan rumors,45.94.47.66,"Nov 4, 2023",barcelona legend lionel messi ruled playing eu...,https://cdn.vox-cdn.com/thumbor/0hxGQT1kDUv6-P...,Barcelona fable lionel messi reign fiddle Euro...
8,https://www.barcablaugranes.com/2023/10/20/239...,Barcelona planning tribute for Lionel Messi in...,38.154.227.167,"Oct 20, 2023",fc barcelonais preparing tribute lionel messi ...,https://cdn.vox-cdn.com/thumbor/sxg0Afi-zeo7wJ...,fc barcelonais set tribute lionel messi either...
9,https://www.barcablaugranes.com/2023/11/2/2394...,Messi picks his favourite goal...and it’s a gr...,185.199.228.220,"Nov 2, 2023",lionel messi named favourite goal picking ball...,https://cdn.vox-cdn.com/thumbor/6TYNbXkcdSxeaF...,lionel messi constitute preferred destination ...


In [13]:
def tokenize_text(text):
    # Tokenize text and add special tokens
    tokens = tokenizer.tokenize(text)
    return tokens

In [14]:
combined_df['tokens'] = combined_df['Content'].apply(tokenize_text)

In [17]:
combined_df.head(10)

,Url,Headline,ProxyIP,Publication Date,Content,Image URL,Augmented_Content,tokens
0,https://www.barcablaugranes.com/2024/1/24/2404...,WATCH: Yamal scores Messi-esque goal for Barca...,185.199.229.156,"Jan 24, 2024",barcelona star lamine yamal scored brilliant g...,https://cdn.vox-cdn.com/thumbor/MDWLmC59VjMp-t...,Barcelona star_topology lamine yamal score sup...,"[barcelona, star, lam, ##ine, ya, ##mal, score..."
1,https://www.barcablaugranes.com/2023/12/20/240...,Barcelona president rules out one-match loan f...,185.199.228.220,"Dec 20, 2023",fc barcelonapresident joan laporta ruled repor...,https://cdn.vox-cdn.com/thumbor/W1gfnSiwmfYZPB...,fc barcelonapresident joan laporta prevail acc...,"[fc, barcelona, ##pres, ##ide, ##nt, joan, lap..."
2,https://www.barcablaugranes.com/2023/12/22/240...,"Barca reunion continues as Suarez joins Messi,...",185.199.231.45,"Dec 22, 2023",luis suarez become latest former barcelona pla...,https://cdn.vox-cdn.com/thumbor/eh4NUh7GSdw_El...,luis suarez become former early Barcelona thes...,"[luis, suarez, become, latest, former, barcelo..."
3,https://www.barcablaugranes.com/2023/12/5/2398...,Lionel Messi confirms returning to Barcelona w...,188.74.183.10,"Dec 5, 2023",lionel messi confirmed going back tofc barcelo...,https://cdn.vox-cdn.com/thumbor/wuXvEFdpfgqbAq...,lionel messi support leaving game tofc barcelo...,"[lionel, mess, ##i, confirmed, going, back, to..."
4,https://www.barcablaugranes.com/2023/12/5/2398...,"Messi has always been my idol, says Lamine Yamal",188.74.210.21,"Dec 5, 2023",barcelona wonderkid lamine yamal regularly lik...,https://cdn.vox-cdn.com/thumbor/c7o6yEkfyg5RQe...,Barcelona wonderkid lamine yamal on_a_regular_...,"[barcelona, wonder, ##ki, ##d, lam, ##ine, ya,..."
5,https://www.barcablaugranes.com/2023/12/1/2398...,Lionel Messi opens up on feud with Robert Lewa...,45.155.68.129,"Dec 1, 2023",barcelona legend lionel messi admitted pretty ...,https://cdn.vox-cdn.com/thumbor/OXccKTnc9uTI6R...,Barcelona caption lionel messi acknowledge som...,"[barcelona, legend, lionel, mess, ##i, admitte..."
6,https://www.barcablaugranes.com/2023/11/14/239...,Barcelona target Messinho shines at U17 World Cup,154.95.36.199,"Nov 14, 2023",estevo willian also known nickname messinho sh...,https://cdn.vox-cdn.com/thumbor/GgY_lYAzUiRiKu...,estevo willian as_well fuck cognomen messinho ...,"[este, ##vo, will, ##ian, also, known, nicknam..."
7,https://www.barcablaugranes.com/2023/11/4/2394...,Messi responds to Barcelona loan rumors,45.94.47.66,"Nov 4, 2023",barcelona legend lionel messi ruled playing eu...,https://cdn.vox-cdn.com/thumbor/0hxGQT1kDUv6-P...,Barcelona fable lionel messi reign fiddle Euro...,"[barcelona, legend, lionel, mess, ##i, ruled, ..."
8,https://www.barcablaugranes.com/2023/10/20/239...,Barcelona planning tribute for Lionel Messi in...,38.154.227.167,"Oct 20, 2023",fc barcelonais preparing tribute lionel messi ...,https://cdn.vox-cdn.com/thumbor/sxg0Afi-zeo7wJ...,fc barcelonais set tribute lionel messi either...,"[fc, barcelona, ##is, preparing, tribute, lion..."
9,https://www.barcablaugranes.com/2023/11/2/2394...,Messi picks his favourite goal...and it’s a gr...,185.199.228.220,"Nov 2, 2023",lionel messi named favourite goal picking ball...,https://cdn.vox-cdn.com/thumbor/6TYNbXkcdSxeaF...,lionel messi constitute preferred destination ...,"[lionel, mess, ##i, named, favourite, goal, pi..."
